In [ ]:
import pandas as pd
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_symptoms = pd.read_csv('test_data/results_symptoms.csv')

In [ ]:
len(df_symptoms)

In [ ]:
df_symptoms.columns

In [ ]:
df_symptoms.head()

In [ ]:
df_symptoms.describe()

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
#df_symptoms.dropna(inplace=True) # leaves 1946

In [ ]:
# normalize the temperaature
#df_symptoms['temperature'] = (df_symptoms['temperature'] - df_symptoms['temperature'].min()) / (df_symptoms['temperature'].max() - df_symptoms['temperature'].min())

In [ ]:
df_symptoms.head(5)

In [ ]:
#df_symptoms[df_symptoms['R'] == 1]
df_symptoms["id"] = df_symptoms.index

In [ ]:
df_symptoms.iloc[:,2:-1].head()

In [ ]:
len(df_symptoms)

In [ ]:
#df_symptoms.head()

In [ ]:
symptom_clf = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', RandomForestClassifier())
    ])

In [ ]:
# don't include the ID as a feature
symptom_clf.fit(df_symptoms.iloc[:,2:-1], df_symptoms['R'])

In [ ]:
symptom_clf['scaler'].scale_

In [ ]:
symptom_clf['clf'].feature_importances_

In [ ]:
feature_df = pd.DataFrame({'feature':df_symptoms.columns[2:-1], 
                           'importance': symptom_clf['clf'].feature_importances_})

In [ ]:
pysqldf("SELECT * FROM feature_df ORDER BY importance DESC")

In [ ]:
symptom_scores = cross_val_score(symptom_clf, df_symptoms.iloc[:,2:], df_symptoms['R'], cv=5)

In [ ]:
print("Accuracy: %0.2f, StDev: %0.2f)" % (symptom_scores.mean(), symptom_scores.std()))

In [ ]:
test_predicted_proba = symptom_clf.predict_proba(df_symptoms.iloc[:, 2:-1])

In [ ]:
test_predicted = symptom_clf.predict(df_symptoms.iloc[:, 2:-1])

In [ ]:
df_pred = pd.DataFrame({"p_cat":test_predicted, "p_prob":[item[1] for item in test_predicted_proba], "p_id":df_symptoms.id })

In [ ]:
pysqldf("SELECT * FROM df_pred ORDER BY p_prob DESC LIMIT 5")

In [ ]:
symptom_clf.predict_proba([[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])

In [ ]:
symptom_clf.predict_proba([[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]])

In [ ]:
symptom_clf.predict_proba([list(df_symptoms.iloc[123][2:-1])])

In [ ]:
symptom_clf.predict_proba([[1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,]])